In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  #for plotting purpose
#from sklearn.preprocessing import linear_mode
import tensorflow as tf

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("JHU_Cases_Deaths_with_CRE_Data.csv")
# Drop the non-beneficial columns
df = df.drop(columns=["Lat","Long","State","POPUNI","Total_Population","Unnamed: 0"])
df

FileNotFoundError: [Errno 2] No such file or directory: 'JHU_Cases_Deaths_with_CRE_Data.csv'

In [ ]:
df.dtypes

## Exploratory Plots

In [ ]:
#INDEPENDANT VARIABLES: "Zero_RF", "One-Two_RF", "Three_RF"
#DEPENDANT VARIABLE: "Deaths" (to be predicted)

In [ ]:
#Zero_RF
plt.subplot(121)
plt.xlabel("Zero_RF")
plt.ylabel("Deaths")
plt.title("Zero_RF -vs- Deaths")
plt.scatter(df["Zero_RF"],df["Deaths"])
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=2.0, hspace=0.25, wspace=0.35)

In [ ]:
#One-Two_RF
plt.subplot(121)
plt.xlabel("One-Two_RF")
plt.ylabel("Deaths")
plt.title("One-Two_RF -vs- Deaths")
plt.scatter(df["One-Two_RF"],df["Deaths"])
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=2.0, hspace=0.25, wspace=0.35)

In [ ]:
#Three_RF
plt.subplot(121)
plt.xlabel("Three_RFF")
plt.ylabel("Deaths")
plt.title("Three_RF -vs- Deaths")
plt.scatter(df["Three_RF"],df["Deaths"])
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=2.0, hspace=0.25, wspace=0.35)

## Training and Building Multivariate Linear Regression Model

In [ ]:
X=df[["Zero_RF", "One-Two_RF", "Three_RF"]]
y=df[["Deaths"]]

In [ ]:
#split data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_L1 = 90
hidden_nodes_L2 = 30
hidden_nodes_L3 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L1, input_dim=number_input_features, activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L2, activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_L3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
#Create a callback that saves the model's weights every 5 epochs.
import os
from tensorflow.keras.callbacks import ModelCheckpoint

callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=0, save_weights_only=True, save_freq=5)
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
nn.save("CREvDeaths_V1.h5")